In [1]:
import pandas as pd
import wandb
from pathlib import Path
import json
import numpy as np

In [2]:
method="procrustes" 
extend_exp="freq"
vm_name = {
    'resnet101': 'ResNet101',
    'resnet152': 'ResNet152',
    'resnet18': 'ResNet18',
    'resnet34': 'ResNet34',
    'resnet50': 'ResNet50',
    'segformer-b0-finetuned-ade-512-512': 'SegFormer<sub>B0</sub>',
    'segformer-b1-finetuned-ade-512-512': 'SegFormer<sub>B1</sub>',
    'segformer-b2-finetuned-ade-512-512': 'SegFormer<sub>B2</sub>',
    'segformer-b3-finetuned-ade-512-512': 'SegFormer<sub>B3</sub>',
    'segformer-b4-finetuned-ade-512-512': 'SegFormer<sub>B4</sub>',
    'segformer-b5-finetuned-ade-640-640': 'SegFormer<sub>B5</sub>',
    'vit-mae-base': 'MAE<sub>Base</sub>',
    'vit-mae-huge': 'MAE<sub>Huge</sub>',
    'vit-mae-large': 'MAE<sub>Large</sub>'
}

model_name = {'bert_uncased_L-2_H-128_A-2': "BERT<sub>TINY</sub>",
                'bert_uncased_L-4_H-256_A-4': "BERT<sub>MINI</sub>" ,
                'bert_uncased_L-4_H-512_A-8': "BERT<sub>SMALL</sub>",
                'bert_uncased_L-8_H-512_A-8': "BERT<sub>MEDIUM</sub>",
                'bert-base-uncased': "BERT<sub>BASE</sub>",
                'bert-large-uncased': "BERT<sub>LARGE</sub>",
                'gpt2': "GPT2<sub>BASE</sub>",
                'gpt2-medium': "GPT2<sub>MEDIUM</sub>",
                'gpt2-large': "GPT2<sub>LARGE</sub>",
                'gpt2-xl': "GPT2<sub>XL</sub>",
                'opt-125m': "OPT<sub>125M</sub>",
                'opt-1.3b': "OPT<sub>1.3B</sub>",
                'opt-6.7b': "OPT<sub>6.7B</sub>",
                'opt-30b': "OPT<sub>30B</sub>",
                'opt-66b': "OPT<sub>66B</sub>",
                'Llama-2-7b-hf': "Llama-2<sub>7B</sub>",
                'Llama-2-13b-hf': "Llama-2<sub>13B</sub>",
                'Llama-2-70b-hf': "Llama-2<sub>70B</sub>",
                "fasttext":"fastText"}

model_size = {'bert_uncased_L-2_H-128_A-2':4.4, 'bert_uncased_L-4_H-256_A-4':11.3, 'bert_uncased_L-4_H-512_A-8':29.1, 
    'bert_uncased_L-8_H-512_A-8':41.7, 'bert-base-uncased':110, 'bert-large-uncased':340, 
    'gpt2':117,  'gpt2-large':762, 'gpt2-xl':1542, 'opt-125m':125, 'opt-6.7b':6700, 'opt-30b':30000,'opt-66b':66000,
    'Llama-2-7b-hf': 7000,
    'Llama-2-13b-hf': 13000,
    'Llama-2-70b-hf': 70000,}

model_tags = {
    "ft": "fasttext",
    "bert": "bert",
    "gpt2": "gpt2",
    "opt": "opt"
}

bins = {
    "_freq10000":"0-10k",
    "_freq100000":"10k-100k",
    "_freq_end":"100k+"
}

bins_order = {
    "_freq10000":"0",
    "_freq100000":"1",
    "_freq_end":"2"
}

set_name = "cleaned"

In [3]:


api = wandb.Api()
df = pd.DataFrame()

for model_alias in ["LM"]:
    # method_name = "Procrustes Analysis" if method == "procrustes" else "Ridge Regression"
    runs = api.runs(path=f"jiaangli00/image2{model_alias}-TACL-{set_name}{extend_exp}")
    # metrics = runs.summary["Results"]

    # data = []
    for i, single_run in enumerate(runs):
        metric = json.load(single_run.file(single_run.summary["Results"]["path"]).download(exist_ok=True))
        df = pd.concat([df,pd.DataFrame(metric["data"], columns=metric["columns"])])

# # df['Subjects'] = df['Subjects'].replace({f'brain_{i}': f'Subject-{i}' for i in range(1, num_subs + 1)})
# df['Layers'] = df['Layers'].replace({f'layer_{i}': f'layer-{i})
df['Models_size'] = df['LM'].copy()
df['Models_size'] = df['Models_size'].replace(model_size)
df['VM'] = df['VM'].replace(vm_name)
df['LM'] = df['LM'].replace(model_name)
df['Bins_order'] = df['Bins'].copy()
df['Bins_order'] = df['Bins_order'].replace(bins_order)
df["Bins"] = df["Bins"].replace(bins)
# #
group_names: list[str] = ["LM", "VM"]
if extend_exp != "":
    group_names.append("Bins")
    group_names.append("Bins_order")
#
precision_csls, precision_nn = [], []
for k in [100]:
    precision_csls.append(f'P@{k}-CSLS')
    precision_nn.append(f'P@{k}-NN')

precision = precision_csls
#
precision.append("Models_size")

In [4]:
seed_avg = df.groupby(group_names)[precision].mean().round(1).reset_index()
_selected = seed_avg.loc[(seed_avg["LM"] == "BERT_LARGE") | (seed_avg["LM"] == "GPT2_XL") | (seed_avg["LM"] == "OPT_30B") | (seed_avg["LM"] == "Llama-2-13B")]
# _selected.to_csv("freq.csv")
# seed_avg
# _selected

In [5]:
table = _selected.loc[(seed_avg["VM"] == "MAE-Huge") | (seed_avg["VM"] == "SegFormer-B5") | (seed_avg["VM"] == "ResNet152") ]

In [6]:
table

,LM,VM,Bins,Bins_order,P@100-CSLS,Models_size


In [7]:
# table.to_csv("freq.csv")

In [8]:
table2 = seed_avg.loc[(seed_avg["VM"] != "MAE<sub>Huge</sub>") & (seed_avg["VM"] != "SegFormer<sub>B5</sub>") & (seed_avg["VM"] != "ResNet152") ]
table2 = table2.sort_values(by=["Bins_order","Models_size"], ascending=[False,True])
table2

,LM,VM,Bins,Bins_order,P@100-CSLS,Models_size
1,BERT<sub>LARGE</sub>,MAE<sub>Base</sub>,100k+,2,34.0,340.0
7,BERT<sub>LARGE</sub>,MAE<sub>Large</sub>,100k+,2,35.1,340.0
10,BERT<sub>LARGE</sub>,ResNet101,100k+,2,43.5,340.0
16,BERT<sub>LARGE</sub>,ResNet18,100k+,2,39.5,340.0
19,BERT<sub>LARGE</sub>,ResNet34,100k+,2,41.6,340.0
...,...,...,...,...,...,...
150,OPT<sub>30B</sub>,SegFormer<sub>B0</sub>,0-10k,0,20.6,30000.0
153,OPT<sub>30B</sub>,SegFormer<sub>B1</sub>,0-10k,0,23.3,30000.0
156,OPT<sub>30B</sub>,SegFormer<sub>B2</sub>,0-10k,0,23.8,30000.0
159,OPT<sub>30B</sub>,SegFormer<sub>B3</sub>,0-10k,0,24.5,30000.0


In [9]:
import plotly.express as px
# import plotly.graph_objects as go

# fig = go.Figure()
traces = []
# df['VM_Bins'] = table2['VM'] + '_' + table2['Bins']
color_scale = px.colors.qualitative.Set3

# Create stacked and grouped bar chart
fig = px.bar(table2, x='VM', y='P@100-CSLS', color='Bins', facet_col="LM",
             labels={'P@100-CSLS': 'P@100'},
            #  facet_col_wrap=3,
            #  category_orders={'VM': ['MAE-Huge', 'ResNet152', 'SegFormer-B5']},
             color_discrete_map={'0-10k': color_scale[5],"10k-100k":color_scale[0],"100k+":color_scale[3]},
             height=500, width=1500
            #  marker="Bins"
            #  facet_col_titles='none'
            #  barmode="group"
             )


# print(fig.data)

for i, trace in enumerate(fig.data):
    trace.marker.opacity = 0.9
    # if ("BERT<sub>LARGE</sub>" or "GPT2<sub>XL</sub>" in trace["hovertemplate"]):
    #     if "Bins=10k-100k" in trace["hovertemplate"]:
    # # trace.marker.color = color_scale[i%3]  # Set color
    #         trace.marker.opacity = 0.7  # Set opacity
    # else:
    #     trace.marker.opacity = 1


# Update layout for better appearance
fig.update_layout(barmode='overlay', showlegend=True,
                  font=dict(size=30),
                  yaxis=dict(title=dict(text='P@100', font=dict(size=30)), tickfont=dict(size=25)),  # Change y-axis font size
                  legend=dict(title=dict(text='Frequency', font=dict(size=30))),
                  xaxis=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis2=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis3=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis4=dict(title=dict(text=''),tickfont=dict(size=20)),
                  template="plotly_white"
                  ) 

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# fig.update_xaxes(title_text='', showgrid=False)

# fig.update_xaxes(title_font=dict(size=20), tickfont=dict(size=20))

# Show the plot
fig.show()
fig.write_image(f"freq_remain_{set_name}_v1.pdf")

In [26]:
import plotly.express as px
# import plotly.graph_objects as go

# fig = go.Figure()
traces = []
# df['VM_Bins'] = table2['VM'] + '_' + table2['Bins']
color_scale = px.colors.qualitative.Set3

# Create stacked and grouped bar chart
fig = px.scatter(table2, x='VM', y='P@100-CSLS', color='Bins', facet_col="LM",
             labels={'P@100-CSLS': 'P@100'},
             color_discrete_map={'0-10k': color_scale[5],"10k-100k":color_scale[0],"100k+":color_scale[3]},
             height=500, width=1600,
             symbol="Bins"
            #  facet_col_titles='none'
            #  barmode="group"
             )

# fig = px.bar_polar(table2, r='P@100-CSLS', theta='VM', color='Bins',
#                    labels={'P@100-CSLS': 'P@100'},
#                    color_discrete_map={'low': color_scale[5], "medium": color_scale[0], "high": color_scale[3]},
#                    height=1000, width=1500,
#                 #    animation_frame='LM',  # If you want to include animation over 'VM'
#                    )


for i, trace in enumerate(fig.data):
    if "Bins=10k-100k" in trace["hovertemplate"]:
    # trace.marker.color = color_scale[i%3]  # Set color
        trace.marker.opacity = 0.5  # Set opacity
    else:
        trace.marker.opacity = 1
    # trace.marker.color = color_scale[i%3]  # Set color
    # trace.marker.opacity = 0.6  # Set opacity
    trace.marker.size=15


# Update layout for better appearance
fig.update_layout(barmode='overlay', showlegend=True,
                  font=dict(size=30),
                  yaxis=dict(title=dict(text='P@100', font=dict(size=30)), tickfont=dict(size=25)),  # Change y-axis font size
                  legend=dict(title=dict(text='Frequency', font=dict(size=30))),
                  xaxis=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis2=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis3=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis4=dict(title=dict(text=''),tickfont=dict(size=20)),
                  template="plotly_white"
                  ) 

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# fig.update_xaxes(title_text='', showgrid=False)

# fig.update_xaxes(title_font=dict(size=20), tickfont=dict(size=20))

# Show the plot
fig.show()
fig.write_image(f"freq_remain_{set_name}_v1.pdf")

In [9]:
for j in ["BERT_LARGE", "GPT2_XL", "OPT_30B", "Llama-2-13B"]:
    print(j)
    for i in ["_freq10000", "_freq100000", "_freq_end"]:
        all_values = table.loc[(_selected["LM"] == j) & (_selected["Bins"] == i), precision].values
        formatted_values = " & ".join(f"{value}" for one_model_value in all_values for value in one_model_value)
        print(f" & " + formatted_values + "\\\\")


BERT_LARGE
 & 23.2 & 25.4 & 24.3\\
 & 34.5 & 44.8 & 40.2\\
 & 35.3 & 43.6 & 40.5\\
GPT2_XL
 & 21.0 & 22.1 & 22.0\\
 & 39.7 & 50.4 & 45.4\\
 & 40.2 & 48.7 & 44.7\\
OPT_30B
 & 24.5 & 24.7 & 25.4\\
 & 47.2 & 56.1 & 52.3\\
 & 55.2 & 64.0 & 59.0\\
Llama-2-13B
 & 18.7 & 21.6 & 18.1\\
 & 40.8 & 52.3 & 46.9\\
 & 53.1 & 64.0 & 57.0\\


In [7]:
a = [2151, 2201, 2199,2184, 2235]
b = [470, 530, 520, 450, 468]
c = [1547, 1628, 1592, 1566, 1609]

print(np.mean(a), np.mean(b), np.mean(c))

2194.0 487.6 1588.4
